## Decision Trees in Practice

In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. You will have to use your solutions from this previous assignment and extend them.

In this assignment you will:

Implement binary decision trees with different early stopping methods.
Compare models with different stopping parameters.
Visualize the concept of overfitting in decision trees.

In [1]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from sklearn import linear_model

In [20]:
loans = pd.read_csv('/Users/kevinwang/Documents/Coursera/Machine Learning - Classification/Week4/lending-club-data.csv')  

/Users/kevinwang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans', axis = 1)

print(loans['emp_length'])

0         10+ years
1          < 1 year
2         10+ years
3         10+ years
4           3 years
            ...    
122602          NaN
122603    10+ years
122604      5 years
122605    10+ years
122606      3 years
Name: emp_length, Length: 122607, dtype: object


In [22]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [23]:
loans = loans[features + [target]]

In [24]:
def one_hot(loans):
    return pd.get_dummies(loans, dummy_na=True)   # We don't need to identify Str variable, it will just transform 'object' var
    # dummy_na=True => When emp_length has 'NA', we do create emp_length_NA column
loans = one_hot(loans)

In [25]:
import json
with open('/Users/kevinwang/Documents/Coursera/Machine Learning - Classification/Week4/module-6-assignment-train-idx.json', 'r') as f: # Reads the list of most frequent words
    train_idx = json.load(f)
with open('/Users/kevinwang/Documents/Coursera/Machine Learning - Classification/Week4/module-6-assignment-validation-idx.json', 'r') as f1: # Reads the list of most frequent words
    validation_idx = json.load(f1)

train_data = loans.iloc[train_idx]
validation_data = loans.iloc[validation_idx]
print(train_data.shape)        #(37224, 13)
print(validation_data.shape)   #(9284, 13)
train_loans_prop = sum(train_data['safe_loans'] == 1)/len(train_data)   #0.503653556845   Means data is very balanced
valid_loans_prop = sum(validation_data['safe_loans'] == 1)/len(validation_data)   #0.496553209823   Means data is very balanced

y_train = train_data['safe_loans']
x_train = train_data.drop('safe_loans', axis=1)

y_valid = validation_data['safe_loans']
x_valid = validation_data.drop('safe_loans', axis=1)

#print(validation_data)

(37224, 29)
(9284, 29)


## Early stopping methods for decision trees

In this section, we will extend the binary tree implementation from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a maximum depth. (set by parameter max_depth).
2. Reached a minimum node size. (set by parameter min_node_size).
3. Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

For the rest of this assignment, we will refer to these three as early stopping conditions 1, 2, and 3.

### Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this. We will alert you when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.

### Early stopping condition 2: Minimum node size

The function reached_minimum_node_size takes 2 arguments:
1. The data (from a node)
2. The minimum number of data points that a node is allowed to split on, min_node_size.

This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the decision_tree_create function. Your code should be analogous to

In [26]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    return len(data)<= min_node_size
    ## YOUR CODE HERE

Quiz question: Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?

Answer: stop

### Early stopping condition 3: Minimum gain in error reduction

The function error_reduction takes 2 arguments:

1. The error before a split, error_before_split.
2. The error after a split, error_after_split.

This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the decision_tree_create function. Your code should be analogous to

In [27]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    return error_before_split - error_after_split
    ## YOUR CODE HERE

Quiz question: Assume an intermediate node has 6 safe loans and 3 risky loans. For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, what should the tree learning algorithm do next?

answer: stop

## Grabbing binary decision tree helper functions from past assignment

Recall from the previous assignment that we wrote a function intermediate_node_num_mistakes that calculates the number of misclassified examples when predicting the majority class. This is used to help determine which feature is best to split on at a given node of the tree. Use your code from the previous assignment. It should be analogous to

In [28]:
def intermediate_node_num_mistakes(labels_in_node):    # it should named 'leaf', not 'node'
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (actual safe loans)
    num_safe = sum(labels_in_node == 1)  
    # Count the number of -1's (actual risky loans)
    num_risk = sum(labels_in_node == -1)                 
    # Return the number of mistakes that the majority classifier makes for training data.
    return min(num_safe, num_risk)  

We then wrote a function best_splitting_feature that finds the best feature to split on given the data and a list of features to consider. Use your code from the previous assignment. It should be analogous to

In [29]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]   # only keep rows where feature = 0
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature] == 1]  # only keep rows where feature = 1
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])            
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
    return best_feature # Return the best feature we found

Finally, recall the function create_leaf from the previous assignment, which creates a leaf node given a set of target values. Use your code from the previous assignment. It should be analogous to

In [30]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }   ## YOUR CODE HERE 
   
    # Count the number of data points that are with actual value +1 and -1 in this node.
    num_ones = len(target_values[target_values == 1])     # keep only row where target values = +1
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1         ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

## Incorporating new early stopping conditions in binary decision tree implementation

Now, you will implement a function that builds a decision tree handling the three early stopping conditions described in this assignment. In particular, you will write code to detect early stopping conditions 2 and 3. You implemented above the functions needed to detect these conditions. 

The 1st early stopping condition, max_depth, was implemented in the previous assignment and you will not need to reimplement this. In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

### Implementing early stopping condition 2: minimum node size:

Step 1: Use the function reached_minimum_node_size that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the min_node_size argument.
Step 2: Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

### Implementing early stopping condition 3: minimum error reduction:

Note: This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction. Recall that classification error is defined as:

classification error  =  # mistakes / # total examples
Step 1: Calculate the classification error before splitting.
Step 2: Calculate the classification error after splitting. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
Step 3: Use the function error_reduction to that you implemented earlier to write an if condition to detect whether the reduction in error is less than the constant provided (min_error_reduction). Don't forget to use that argument.
Step 4: Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.
Your code should be analogous to

In [31]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print("--------------------------------------------------------------------")
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print("Stopping condition 1 reached. All data points have the same target value.")               
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print("Stopping condition 2 reached. No remaining features.")                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print("Early stopping condition 1 reached. Reached maximum depth.")
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  reached_minimum_node_size(data, min_node_size):        ## YOUR CODE HERE 
        print("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)  ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)    
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]     
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes =    intermediate_node_num_mistakes(left_split[target])
    right_mistakes =   intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:       ## YOUR CODE HERE
        print("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values)  ## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"    If all data goes to one side
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(right_split[target])

    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction) 
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

## Build a tree!

Now that your code is working, train a tree model on the train_data with

max_depth = 6

min_node_size = 100,

min_error_reduction = 0.0

Warning: This code block may take a minute to learn. Call this model my_decision_tree_new. Your code should be analogous to

In [35]:
features_new = list(x_train.columns)        # Since we did one-hot, so can not just use 'feature'
#features_new = list(x_valid.columns)
#print(features_new1)
#print(features_new)
my_decision_tree_new = decision_tree_create(train_data, features_new, 'safe_loans', 0, max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_nan. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (5 data points).
Early stopping condition 2 reached. Reached minimum node size.
-------------------------------------------

Let's now train a tree model ignoring early stopping conditions 2 and 3 so that we get the same tree as in the previous assignment. To ignore these conditions, we set min_node_size=0 and min_error_reduction=-1 (a negative value). Call this model my_decision_tree_old. Your code should be analogous to

In [36]:
my_decision_tree_old = decision_tree_create(train_data, features_new, 'safe_loans', 0, max_depth = 6, 
                                            min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

## Making predictions

Recall that in the previous assignment you implemented a function classify to classify a new point x using a given tree. We will need that function here. Use your code from the previous assignment. It should be analogous to

In [37]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]  # x value of the test point for that splitting feature
        if annotate:
             print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:                        # If the value for that feature = 0,
            return classify(tree['left'], x, annotate)      # if x = 0, go left
        else:
            return classify(tree['right'], x, annotate)     # if x = 1, go right

Now, let's consider the first example of the validation set and see what the my_decision_tree_new model predicts for this data point. Your code should be analogous to

In [38]:
#print validation_set[0]
#print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_set[0])

print(validation_data.iloc[0,:])
type(validation_data.iloc[0,:])   #series
print('Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0,:]))

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
grade_nan                  0
term_ 36 months            0
term_ 60 months            1
term_nan                   0
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
home_ownership_nan         0
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_nan             0
Name: 24, dtype: int64
Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [39]:
classify(my_decision_tree_new, validation_data.iloc[0,:], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
At leaf, predicting -1


-1

Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as my_decision_tree_old.

In [40]:
classify(my_decision_tree_old, validation_data.iloc[0,:], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for validation_set[0] shorter, longer, or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

Answer: shorter

Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

Answer: I guess 'shorter or the same'

Quiz question: For a tree trained on any dataset using max_depth = 6, min_node_size = 100, min_error_reduction=0.0, what is the maximum number of splits encountered while making a single prediction?

Answer: 6

## Evaluating the model

Now let us evaluate the model that we have trained. You implemented this evaluation in the function evaluate_classification_error from the previous assignment. Use your code from the previous assignment. It should be analogous to

In [41]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    return sum(prediction != data[target])/float(len(data))

Now, let's use this function to evaluate the classification error of my_decision_tree_new on the validation_set. Your code should be analogous to

In [42]:
print(evaluate_classification_error(my_decision_tree_new, validation_data))

0.38367083153813014


Now, evaluate the validation error using my_decision_tree_old.

In [43]:
print(evaluate_classification_error(my_decision_tree_old, validation_data))

0.3837785437311504


Quiz question: Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?

Answer: lower

## Exploring the effect of max_depth
We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (too small, just right, and too large).

Train three models with these parameters:

model_1: max_depth = 2 (too small)

model_2: max_depth = 6 (just right)

model_3: max_depth = 14 (may be too large)
    
For each of these three, set min_node_size = 0 and min_error_reduction = -1. Make sure to call the models model_1, model_2, and model_3.

Note: Each tree can take up to a few minutes to train. In particular, model_3 will probably take the longest to train.

Let us evaluate the models on the train and validation data. Let us start by evaluating the classification error on the training data. Your code should be analogous to:

In [44]:
model_1 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 2, 
                                            min_node_size = 0, min_error_reduction=-1)
model_2 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 0, min_error_reduction=-1)
model_3 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 14, 
                                            min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade_D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Early stopping condition 1 reached. Reached maximum depth.
-----------------------------------------------

Split on feature grade_F. (2133, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 5 (2058 data points).
Split on feature grade_E. (2058, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (2190 data points).
Split on feature grade_D. (2190, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Split on feature emp_length_5 years. (969, 79)
--------------------------------------------------------------------
Subtree, depth = 4 (969 data points).
Split on feature grade_C. (969, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (79 data points).
Split on feature home_ownership_MORTGAGE. (34, 45)
--------------------------------------------------------------------
Subtree, depth = 5 (34 data points).
Split on feature grade_C. (34, 0)
Cr

Split on feature grade_A. (702, 230)
--------------------------------------------------------------------
Subtree, depth = 6 (702 data points).
Split on feature home_ownership_OTHER. (701, 1)
--------------------------------------------------------------------
Subtree, depth = 7 (701 data points).
Split on feature grade_B. (317, 384)
--------------------------------------------------------------------
Subtree, depth = 8 (317 data points).
Split on feature grade_C. (1, 316)
--------------------------------------------------------------------
Subtree, depth = 9 (1 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 9 (316 data points).
Split on feature grade_G. (316, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 8 (384 data points).
Split on feature grade_C. (384, 0)
Creating leaf node.
-------------------

Now evaluate the classification error on the validation data.

In [45]:
print("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data))
print("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data))
print("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data))

print("Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data))
print("Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data))
print("Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data))

Training data, classification error (model 1): 0.40003761014399314
Training data, classification error (model 2): 0.38185041908446166
Training data, classification error (model 3): 0.3761820330969267
Validation data, classification error (model 1): 0.3981042654028436
Validation data, classification error (model 2): 0.3837785437311504
Validation data, classification error (model 3): 0.37731581214993537


Quiz Question: Which tree has the smallest error on the validation data?

Answer: model_3

Quiz Question: Does the tree with the smallest error in the training data also have the smallest error in the validation data?

Answer: Yes, model_3

Quiz Question: Is it always true that the tree with the lowest classification error on the training set will result in the lowest classification error in the validation set?

Answer: No

## Measuring the complexity of the tree
Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

complexity(T) = number of leaves in the tree T

Here, we provide a function count_leaves that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in model_1, model_2, and model_3. This code is in Python. If you are using another language, make sure your code is analogous to

In [46]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

Using the function count_leaves, compute the number of nodes in model_1, model_2, and model_3.

In [47]:
print(count_leaves(model_1))
print(count_leaves(model_2))
print(count_leaves(model_3))

4
19
41


Quiz question: Which tree has the largest complexity?
    
Answer: Model3

Quiz question: Is it always true that the most complex tree will result in the lowest classification error in the validation_set?

Answer: No

## Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (negative, just right, and too positive).

Train three models with these parameters:

model_4: min_error_reduction = -1 (ignoring this early stopping condition)

model_5: min_error_reduction = 0 (just right)

model_6: min_error_reduction = 5 (too positive)

For each of these three, we set max_depth = 6, and min_node_size = 0. Make sure to call the models model_4, model_5, 
and model_6.

Note: Each tree can take up to 30 seconds to train.

Calculate the accuracy of each model (model_4, model_5, or model_6) on the validation set. Your code should be analogous to

In [48]:
model_4 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 0, min_error_reduction=-1)
model_5 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 0, min_error_reduction=0)
model_6 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Subtree, depth = 4 (358 data points).
Split on feature emp_length_8 years. (347, 11)
--------------------------------------------------------------------
Subtree, depth = 5 (347 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 5 (11 data points).
Split on feature home_ownership_OWN. (9, 2)
--------------------------------------------------------------------
Subtree, depth = 6 (9 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (2 data points).
Stopping condition 1 reached. All data points have the same target value.
--------------------------------------------------------------------
Subtree, depth = 3 (1276 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth =

In [49]:
print("Training data, classification error (model 4):", evaluate_classification_error(model_4, train_data))
print("Training data, classification error (model 5):", evaluate_classification_error(model_5, train_data))
print("Training data, classification error (model 6):", evaluate_classification_error(model_6, train_data))

print("Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data))
print("Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data))
print("Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data))

Training data, classification error (model 4): 0.38185041908446166
Training data, classification error (model 5): 0.3819578766387277
Training data, classification error (model 6): 0.4963464431549538
Validation data, classification error (model 4): 0.3837785437311504
Validation data, classification error (model 5): 0.3837785437311504
Validation data, classification error (model 6): 0.503446790176648


Using the count_leaves function, compute the number of leaves in each of each models in (model_4, model_5, and model_6).

In [50]:
print(count_leaves(model_4))
print(count_leaves(model_5))
print(count_leaves(model_6))

19
13
1


Quiz Question: Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?

Answer: model4

Quiz Question: model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?

Answer: Yes

## Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (too small, just right, and just right).

Train three models with these parameters:

model_7: min_node_size = 0 (too small)

model_8: min_node_size = 2000 (just right)

model_9: min_node_size = 50000 (too large)

For each of these three, we set max_depth = 6, and min_error_reduction = -1. Make sure to call these models model_7, model_8, and model_9.

Note: Each tree can take up to 30 seconds to train.

Calculate the accuracy of each model (model_7, model_8, or model_9) on the validation set.

Using the count_leaves function, compute the number of leaves in each of each models (model_7, model_8, and model_9).

In [51]:
model_7 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 0, min_error_reduction=-1)
model_8 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 2000, min_error_reduction=-1)
model_9 = decision_tree_create(train_data, features_new, 'safe_loans', max_depth = 6, 
                                            min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
E

Split on feature emp_length_nan. (20734, 932)
--------------------------------------------------------------------
Subtree, depth = 5 (20734 data points).
Split on feature grade_G. (20638, 96)
--------------------------------------------------------------------
Subtree, depth = 6 (20638 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 6 (96 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 5 (932 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 4 (358 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (1276 data points).
Early stopping condition 2

In [52]:
print("Training data, classification error (model 7):", evaluate_classification_error(model_7, train_data))
print("Training data, classification error (model 8):", evaluate_classification_error(model_8, train_data))
print("Training data, classification error (model 9):", evaluate_classification_error(model_9, train_data))

print("Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data))
print("Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data))
print("Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data))

Training data, classification error (model 7): 0.38185041908446166
Training data, classification error (model 8): 0.3837040618955513
Training data, classification error (model 9): 0.4963464431549538
Validation data, classification error (model 7): 0.3837785437311504
Validation data, classification error (model 8): 0.38453252908229213
Validation data, classification error (model 9): 0.503446790176648


In [53]:
print(count_leaves(model_7))
print(count_leaves(model_8))
print(count_leaves(model_9))

19
12
1


Quiz Question: Using the results obtained in this section, which model (model_7, model_8, or model_9) would you choose to use?

Answer: model8   